In [20]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from tensorflow import keras
from sklearn import preprocessing
import keras.backend as K

#precision & recall by: https://github.com/keras-team/keras/issues/5400
#Code Source: https://cloud.google.com/blog/products/gcp/intro-to-text-classification-with-keras-automatically-tagging-stack-overflow-posts

trainDF = pd.concat([pd.read_pickle('../../data/raw_data/MasterData_2015.pkl.gz'),
                    pd.read_pickle('../../data/raw_data/MasterData_2014.pkl.gz'),
                    pd.read_pickle('../../data/raw_data/MasterData_2013.pkl.gz'),
                    pd.read_pickle('../../data/raw_data/MasterData_2012.pkl.gz')])
                                   
trainDF = trainDF[trainDF.TEXT.notna() & trainDF.NTEE.notna()]
trainDF['text'] = trainDF['TEXT'].astype(str)
trainDF['label'] = trainDF['NTEE'].astype(str)

trainDF = trainDF.drop(['NTEE', 'IRS_URL', 'TEXT'], axis=1)


"\ntrain_df, test_df = np.split(trainDF, [int(.7*len(trainDF))])\n\n#tf.logging.set_verbosity(tf.logging.ERROR)\n\ntrain_posts = train_df['text']\ntrain_tags = train_df['label']\ntest_posts = test_df['text']\ntest_tags = test_df['label']\nvocab_size = 1000\ntokenize = keras.preprocessing.text.Tokenizer(num_words=vocab_size)\ntokenize.fit_on_texts(train_posts)\n\nx_train = tokenize.texts_to_matrix(train_posts)\nx_test = tokenize.texts_to_matrix(test_posts)\n\nencoder = preprocessing.LabelBinarizer()\nencoder.fit(train_tags)\ny_train = encoder.transform(train_tags)\ny_test = encoder.transform(test_tags)\n"

In [4]:
import statistics

counts = trainDF['label'].value_counts().sort_index().to_frame()
counts['category'] = counts.index
counts['train_sample']=(counts['label']/2).astype(int)

def dataformodel(trainDF):
    
    train_df, test_df = np.split(trainDF, [int(.7*len(trainDF))])

    #tf.logging.set_verbosity(tf.logging.ERROR)

    train_posts = train_df['text']
    train_tags = train_df['label']
    test_posts = test_df['text']
    test_tags = test_df['label']
    vocab_size = 1000
    tokenize = keras.preprocessing.text.Tokenizer(num_words=vocab_size)
    tokenize.fit_on_texts(train_posts)

    x_train = tokenize.texts_to_matrix(train_posts)
    x_test = tokenize.texts_to_matrix(test_posts)

    encoder = preprocessing.LabelBinarizer()
    encoder.fit(train_tags)
    y_train = encoder.transform(train_tags)
    y_test = encoder.transform(test_tags)
    
    def precision(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    num_labels=26
    batch_size = 500
    epochs = 40

    model = keras.Sequential()
    model.add(keras.layers.Dense(512, input_shape=(vocab_size,)))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Dense(num_labels))
    model.add(keras.layers.Activation('softmax'))

    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  #metrics=['accuracy'],
                 metrics=['binary_accuracy', precision, recall])

    history = model.fit(x_train, y_train, 
                        batch_size=batch_size, 
                        epochs=epochs, 
                        verbose=1, 
                        validation_split=0.1)
    
    score = model.evaluate(x_test, y_test, 
                       batch_size=batch_size, verbose=1)
    return score


result = dataformodel(trainDF)
print(result)

Train on 328851 samples, validate on 36539 samples
Epoch 1/40
328851/328851 [==============================] - 23s 70us/step - loss: 1.5055 - binary_accuracy: 0.9731 - precision: 0.7669 - recall: 0.4167 - val_loss: 1.2856 - val_binary_accuracy: 0.9758 - val_precision: 0.7970 - val_recall: 0.4961
Epoch 2/40
328851/328851 [==============================] - 23s 69us/step - loss: 1.2499 - binary_accuracy: 0.9763 - precision: 0.7968 - recall: 0.5140 - val_loss: 1.1864 - val_binary_accuracy: 0.9773 - val_precision: 0.8053 - val_recall: 0.5400
Epoch 3/40
328851/328851 [==============================] - 25s 77us/step - loss: 1.1350 - binary_accuracy: 0.9781 - precision: 0.8163 - recall: 0.5540 - val_loss: 1.1053 - val_binary_accuracy: 0.9787 - val_precision: 0.8194 - val_recall: 0.5733
Epoch 4/40
328851/328851 [==============================] - 24s 73us/step - loss: 1.0223 - binary_accuracy: 0.9800 - precision: 0.8386 - recall: 0.5949 - val_loss: 1.0272 - val_binary_accuracy: 0.9803 - val_prec

Epoch 34/40
328851/328851 [==============================] - 21s 65us/step - loss: 0.1827 - binary_accuracy: 0.9967 - precision: 0.9732 - recall: 0.9404 - val_loss: 0.8195 - val_binary_accuracy: 0.9904 - val_precision: 0.8947 - val_recall: 0.8504
Epoch 35/40
328851/328851 [==============================] - 23s 70us/step - loss: 0.1797 - binary_accuracy: 0.9968 - precision: 0.9734 - recall: 0.9417 - val_loss: 0.8192 - val_binary_accuracy: 0.9903 - val_precision: 0.8897 - val_recall: 0.8534
Epoch 36/40
328851/328851 [==============================] - 23s 69us/step - loss: 0.1786 - binary_accuracy: 0.9968 - precision: 0.9734 - recall: 0.9423 - val_loss: 0.8299 - val_binary_accuracy: 0.9905 - val_precision: 0.8942 - val_recall: 0.8527
Epoch 37/40
328851/328851 [==============================] - 20s 62us/step - loss: 0.1766 - binary_accuracy: 0.9968 - precision: 0.9737 - recall: 0.9428 - val_loss: 0.8368 - val_binary_accuracy: 0.9905 - val_precision: 0.8954 - val_recall: 0.8514
Epoch 38/40


In [45]:
from tqdm import tqdm_notebook as tqdm

all_pred = pd.DataFrame(columns=['EIN', 'text', 'actual_label', 'predicted_label', 'true_pred'])


for i in tqdm(range(50947, len(x_test))):
    prediction = model.predict(np.array([x_test[i]]))    
    text_labels = encoder.classes_ 
    predicted_label = text_labels[np.argmax(prediction[0])]
    text = test_posts.iloc[i]
    ein = trainDF[trainDF['text']==test_posts.iloc[i]]['EIN'].astype(int).drop_duplicates().tolist()
    actual_label = trainDF[trainDF['text']==test_posts.iloc[i]]['label'].drop_duplicates().tolist()
    if(predicted_label == test_tags.iloc[i]):
        true_pred = 'true'
    else:
        true_pred = 'false'

    all_pred.loc[len(all_pred)] = [ein, text, actual_label, predicted_label, true_pred]


if(os.path.exists('../../data/classification_results/all_predictions.pkl.gz')):
    all_pred = pd.concat([pd.read_pickle('../../data/classification_results/all_predictions.pkl.gz'), all_pred]).drop_duplicates()
    
#all_pred.to_pickle('../../data/classification_results/all_predictions.pkl.gz', index=False)


TypeError: unhashable type: 'list'